# Assigning Regions to Sale Representatives at Pfizer Turkey 


In [126]:
import pandas as pd
import numpy as np
from ortools.linear_solver import pywraplp
import json
import collections

assignment = pd.read_csv("assignment.csv")
distances = pd.read_csv("brick_rp_distances.csv")
workload = pd.read_csv("bricks_index_values.csv")

In [127]:
def str_to_list(to_cast):
    return json.loads(to_cast)

In [135]:
def create_init_assign_matrix(assignments):
    assignment_d = {}
    for i in range(4):
        x = assignment[assignment["SR#"] == (i+1)]
        for brick in str_to_list(x["Bricks_Assigned"][i]):
            assignment_d[brick] = i + 1
    sorted_dict = dict(sorted(assignment_d.items()))
    matrix = [[int((i+1) == sorted_dict[k])for i in range(4)] for k in sorted_dict.keys()]
    return matrix

In [136]:
def create_index_value_matrix(bricks_index_values):
    index_dict = bricks_index_values.set_index("brick").T.to_dict('list')
    sorted_dict = dict(sorted(index_dict.items()))
    return [[float(sorted_dict[index][0]), float(sorted_dict[index][0]), float(sorted_dict[index][0]), float(sorted_dict[index][0])] for index in sorted_dict.keys()]

In [137]:
def create_distances_matrix(distances):
    distances_dict = distances.set_index("brick").T.to_dict('list')
    sorted_dict = dict(sorted(distances_dict.items()))
    return [sorted_dict[index] for index in sorted_dict.keys()]

In [249]:
# Create the mip solver with the CP-SAT backend.
solver = pywraplp.Solver.CreateSolver("SCIP")
if not solver:
    print("Problème lors de la création du solveur")

# Values for constraints
num_sr = 4
num_bricks = len(distances_matrix)
min_workload = 0.8
max_workload = 1.2
init_state_matrix = create_init_assign_matrix(assignment)
index_value_matrix = create_index_value_matrix(workload)
distances_matrix = create_distances_matrix(distances)
hq_bricks = [3,13,15,21]

# Boolean variables
x = {}
for brick in range(num_bricks):
    for sr in range(num_sr):
        x[brick, sr] = solver.BoolVar(f"x[{brick},{sr}]")

# Add constraints
# One brick to one SR and one SR to one brick
for brick in range(num_bricks):
    solver.Add(solver.Sum([x[brick, sr] for sr in range(num_sr)]) == 1)

# The cental brick for an SR cannot change
for sr in range(num_sr):
    solver.Add(solver.Sum(x[hq, sr] for hq in hq_bricks) == 1)

# Balance workload (min and max)
for sr in range(num_sr):
    solver.Add(solver.Sum([index_value_matrix[brick][sr] * x[brick,sr] for brick in range(num_bricks)]) >= min_workload)
    solver.Add(solver.Sum([index_value_matrix[brick][sr] * x[brick,sr] for sr in range(num_sr)]) <= max_workload)

In [250]:
# Minimizing the distance
objective_terms = []
for brick in range(num_bricks):
    for sr in range(num_sr):
        objective_terms.append(distances_matrix[brick][sr] * x[brick, sr])
solver.Minimize(solver.Sum(objective_terms))

In [251]:
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()
print(status)

Solving with SCIP 9.0.0 [LP solver: Glop 9.11]
0


In [252]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for brick in range(num_bricks):
        for sr in range(num_sr):
            if x[brick, sr].solution_value() > 0.5:
                print(
                    f"Brick {brick} assigned to SR {sr}."
                    + f" Cost: {distances_matrix[brick][sr]}"
                )
else:
    print("No solution found.")


Total cost = 154.08

Brick 0 assigned to SR 3. Cost: 21.12
Brick 1 assigned to SR 3. Cost: 17.33
Brick 2 assigned to SR 3. Cost: 12.25
Brick 3 assigned to SR 0. Cost: 0.0
Brick 4 assigned to SR 0. Cost: 3.07
Brick 5 assigned to SR 0. Cost: 1.22
Brick 6 assigned to SR 0. Cost: 2.8
Brick 7 assigned to SR 0. Cost: 2.87
Brick 8 assigned to SR 0. Cost: 3.8
Brick 9 assigned to SR 2. Cost: 4.37
Brick 10 assigned to SR 2. Cost: 2.97
Brick 11 assigned to SR 0. Cost: 21.99
Brick 12 assigned to SR 1. Cost: 3.28
Brick 13 assigned to SR 1. Cost: 0.0
Brick 14 assigned to SR 2. Cost: 1.11
Brick 15 assigned to SR 2. Cost: 0.0
Brick 16 assigned to SR 2. Cost: 1.08
Brick 17 assigned to SR 1. Cost: 0.77
Brick 18 assigned to SR 0. Cost: 11.13
Brick 19 assigned to SR 0. Cost: 17.49
Brick 20 assigned to SR 3. Cost: 25.43
Brick 21 assigned to SR 3. Cost: 0.0
